In [ ]:
!wget https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv

--2022-02-17 09:42:41--  https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60302 (59K) [text/plain]
Saving to: ‘titanic.csv’

titanic.csv         100%[===================>]  58.89K  --.-KB/s    in 0.01s   

2022-02-17 09:42:41 (4.63 MB/s) - ‘titanic.csv’ saved [60302/60302]



In [ ]:
!ls


sample_data  titanic.csv


In [ ]:
import numpy as np 
import pandas as pd

data = pd.read_csv(r'titanic.csv')

data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
m=len(data.axes[0])
n=len(data.axes[1])

In [ ]:
ages = ["minor", "major"]
x=-1
for i in range(m):
  if not pd.isna(data.iloc[i, 5]):
    #print(data.iloc[i, 5])
    data.iloc[i,5] = int(data.iloc[i,5])
for i in range(m):
  if data.iloc[i, 5] <= 18:
    #print("111")
    x = 0
  else:
    x = 1
  data.iloc[i, 5] = ages[x]

In [ ]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,major,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,major,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,major,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,major,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,major,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,major,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,major,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,major,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,major,0,0,111369,30.0000,C148,C


In [ ]:
data.drop(['Name','Fare','Cabin','PassengerId','Ticket'],inplace=True,axis=1)

In [ ]:
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,male,major,1,0,S
1,1,1,female,major,1,0,C
2,1,3,female,major,0,0,S
3,1,1,female,major,1,0,S
4,0,3,male,major,0,0,S
...,...,...,...,...,...,...,...
886,0,2,male,major,0,0,S
887,1,1,female,major,0,0,S
888,0,3,female,major,1,2,S
889,1,1,male,major,0,0,C


In [ ]:
cols=list(data.columns)
examples = [tuple(x) for x in data[cols].to_numpy()]

In [ ]:
def get_domains(examples):
    d = [set() for i in examples[0]]
    for x in examples:
        for i, xi in enumerate(x):
            d[i].add(xi)
    return d

get_domains(examples)

[{0, 1},
 {1, 2, 3},
 {'female', 'male'},
 {'major', 'minor'},
 {0, 1, 2, 3, 4, 5, 8},
 {0, 1, 2, 3, 4, 5, 6},
 {'C', 'Q', 'S', nan}]

In [ ]:
def candidateElimination(attr, outcome): 
    specific_hypothesis = attr[0].copy()
    print("\nInitialization of specific_hypothesis and general_hypothesis")
    print("\nSpecific Boundary: ", specific_hypothesis)
    general_hypothesis = [["?" for i in range(len(specific_hypothesis))] for i in range(len(specific_hypothesis))]
    print("\nGeneric Boundary: ",general_hypothesis)  

    for i, h in enumerate(attr):
        print("\nInstance", i+1 , "is ", h)
        if outcome[i] == 1: 
            print("Instance is Positive ")
            for x in range(len(specific_hypothesis)): 
                if h[x]!= specific_hypothesis[x]:                    
                    specific_hypothesis[x] ='?'                     
                    general_hypothesis[x][x] ='?'
                   
        if outcome[i] == 0: 
            print("Instance is Negative ")  
            for x in range(len(specific_hypothesis)): 
                if h[x]!= specific_hypothesis[x]:                    
                    general_hypothesis[x][x] = specific_hypothesis[x]                
                else:                    
                    general_hypothesis[x][x] = '?'        
        
        print("Specific Boundary after ", i+1, "Instance is ", specific_hypothesis)         
        print("Generic Boundary after ", i+1, "Instance is ", general_hypothesis)
        print("\n")

    final_gh = []
    for i in range(len(general_hypothesis)):
      all_qm = 1
      for j in general_hypothesis[i]:
        if(j != '?'):
          all_qm =0
      if all_qm == 0:
        #print(general_hypothesis[i])
        final_gh.append(general_hypothesis[i])

    #print(final_gh)

    return specific_hypothesis, final_gh 

In [ ]:
def findS(attr, outcome, data):
  specific_hypothesis = attr[0]
  for i in range(len(outcome)):
    if outcome[i] == 1: 
      specific_hypothesis = attr[i].copy()
      break

  print("The intial hypothesis is:", specific_hypothesis)

  for i in range(len(outcome)):
    if outcome[i] == 1:
      for x in range(len(specific_hypothesis)):
        if attr[i][x] != specific_hypothesis[x]:
          specific_hypothesis[x] = '?'
        else:
          pass
  print("\nThe final hypothesis is:",specific_hypothesis)
  return specific_hypothesis

In [ ]:
def matches(test_data,specific,general):
  total_not_pass=0
  x=len(specific) + len(general)
  for i in range(len(test_data)):
    if specific[i] != test_data[i] and specific[i] != '?':
      total_not_pass += 1
      break
  for i in range(len(general)):
    for j in range(len(test_data)):
      if general[i][j] != test_data[j] and general[i][j] != '?':
        total_not_pass += 1
        break
  if total_not_pass / x > 0.5 :
    return False 
  return True
    
        

In [ ]:
def test_accuracy(test_data,test_target,specific,general):
  count = 0
  for i in range(test_data.shape[0]):
    if matches(test_data[i],specific,general) == test_target[i]:
      count = count + 1
  print("The accuracy of candidate elimination is",count/len(test_data)*100)


In [ ]:
def find_accuracy(test_data,test_target,specific):
  count = 0
  for i in range(len(test_data)):
    e=0
    for j in range(len(test_data[i])):
      if specific[j]!= test_data[i][j] and specific[j] != '?' and test_target[i] == 1:
        e=1
        break
    if e == 1:
      count = count + 1
  print("The accuracy of findS is",count/len(test_data)*100)
  



In [ ]:
dat = data.sample(frac=1) # shuffling the dataset
dt = dat[:5] # selecting first 4 as the dataset
# print(dt)
# print("\n")
attr = np.array(dt)[:,2:] # all col except survived
#print(attr)
print('\n')

outcome = np.array(dt)[:,0] #survived col
#print(outcome)
specific_hypothesis, general_hypothesis = candidateElimination(attr, outcome)

print("Final Specific_hypothesis: ", specific_hypothesis, sep="\n")
print("Final general_hypothesis: ", general_hypothesis, sep="\n")




Initialization of specific_hypothesis and general_hypothesis

Specific Boundary:  ['female' 'major' 1 2 'S']

Generic Boundary:  [['?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?']]

Instance 1 is  ['female' 'major' 1 2 'S']
Instance is Positive 
Specific Boundary after  1 Instance is  ['female' 'major' 1 2 'S']
Generic Boundary after  1 Instance is  [['?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?']]



Instance 2 is  ['male' 'major' 0 0 'S']
Instance is Negative 
Specific Boundary after  2 Instance is  ['female' 'major' 1 2 'S']
Generic Boundary after  2 Instance is  [['female', '?', '?', '?', '?'], ['?', '?', '?', '?', '?'], ['?', '?', 1, '?', '?'], ['?', '?', '?', 2, '?'], ['?', '?', '?', '?', '?']]



Instance 3 is  ['male' 'major' 0 0 'S']
Instance is Negative 
Specific Boundary after  3 Instance is  ['female' 'm

In [ ]:
find_specific_hypothesis = findS(attr, outcome,dat)


The intial hypothesis is: ['female' 'major' 1 2 'S']

The final hypothesis is: ['female' 'major' 1 2 'S']


In [ ]:
test = dat[-7:]

test_data = np.array(test)[:,2:]
test_target = np.array(test)[:,0]
find_accuracy(test_data,test_target,find_specific_hypothesis)
test_accuracy(test_data,test_target,specific_hypothesis,general_hypothesis)

The accuracy of findS is 57.14285714285714
The accuracy of candidate elimination is 71.42857142857143


FindS algorithm utilises this general-to-specific ordering,performing a specific-to-general search through the hypothesis space along one branch of the partial ordering, to find the most specific hypothesis consistent with only POSITIVE training examples.
Whereas The CANDIDATE-ELIMINATION algorithm utilises this general-to specific ordering to compute the version space by incrementally computing the sets of maximally specific (S) and maximally general (G) hypotheses consistent with POSITIVE as well as NEGATIVE training examples.
Therefore we can conclude that the candidate elimination algorithm may have more accuracy than the findS algorithm since it classifies positive as well as negative examples.
Thus I learnt the Candidate elimination algorithm and how it is different from FindS.


